In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from numpy.random import choice
from tqdm import tqdm
import time

from numpy.random import random

import pandas as pd

from routines_tab import allGates, gf2RowRed

In [ ]:
def lst_gen(lst):
    for item in lst:
        yield item

In [ ]:
allGates_phaseless = np.empty((len(allGates), 4, 4), dtype='int')
for gate_ind in range(allGates.shape[0]):
    allGates_phaseless[gate_ind] = allGates[gate_ind, [8,4,2,1], :-1].T

In [ ]:
L = 32
timesteps = 10
reps = 500

circuits = choice(np.arange(576,len(allGates)), (L,reps))

for circuit_ind in tqdm(range(reps)):
    floquet_array = circuits[:,circuit_ind]

    pauli_strings = np.identity(2*L, dtype='int8')

    for t in range(timesteps):
        floquet = lst_gen(floquet_array)
        for i in range(0, L ,2):
            gate = allGates[next(floquet)]
            twoQubitClif(pauli_strings, gate, i, i+1)
        for i in range(1, L+1, 2):
            gate = allGates[next(floquet)]
            twoQubitClif(pauli_strings, gate, i, (i+1)%L)
                
    
    pauli_strings_ = np.identity(2*L, dtype='int8')
    floquet = lst_gen(floquet_array)
    for i in range(0, L ,2):
        gate = allGates[next(floquet)]
        twoQubitClif(pauli_strings_, gate, i, i+1)
    for i in range(1, L+1, 2):
        gate = allGates[next(floquet)]
        twoQubitClif(pauli_strings_, gate, i, (i+1)%L)
    pauli_strings_ = np.linalg.matrix_power(pauli_strings_, timesteps) % 2
    
    if not np.array_equal(pauli_strings, pauli_strings_):
        raise Exception()

In [ ]:
def twoQubitClif(tab, gate, a, b):
    """
    PHASELESS gate evolution
    this varies from the usual twoQubitClif for tableaus, here each COLUMN is a stabiliser (Pauli string)
    """
    n = np.shape(tab)[1] // 2
    matches = tab[n+b,:] + 2*tab[n+a,:] + 4*tab[b,:] + 8*tab[a,:]
    #print(a,b,matches)
    tab[a,:] = gate[matches,0] # xa
    tab[b,:] = gate[matches,1] # xb
    tab[n+a,:] = gate[matches,2] # za
    tab[n+b,:] = gate[matches,3] # zb
    
def twoQubitClif_ps(ps, gate, a, b):
    """
    PHASEFUL gate evolution
    acts on 1D array pauli string ps, where the last position indexes the phsae 
    """
    L = len(ps) // 2
    match = ps[L+b] + 2*ps[L+a] + 4*ps[b] + 8*ps[a]
    ps[a] = gate[match,0] # xa
    ps[b] = gate[match,1] # xb
    ps[L+a] = gate[match,2] # za
    ps[L+b] = gate[match,3] # zb
    ps[2*L] = (ps[2*L] + gate[match,4]) % 2 # phase

In [ ]:
def exp_sff(Lsys, Lbath, p, reps, max_t, directory=None, name_ext=None):
    """
    let system and bath be each obc for simplicity
    """
    
    n_evecs_t = np.zeros((max_t,reps))
    allphase1 = np.full((max_t,reps), True)

    L = Lsys + Lbath
    
    for rep in tqdm(range(reps)):
        
        pauli_strings = np.identity(2*L, dtype='int8')
        
        for i in range(0, Lsys-1, 2):
            twoQubitClif(pauli_strings, allGates[choice(np.arange(576,len(allGates)))], i, i+1)
        for i in range(Lsys, L-1, 2):
            twoQubitClif(pauli_strings, allGates[choice(np.arange(576,len(allGates)))], i, i+1)
        for a in range(Lsys):
            if random() > p: continue
            b = choice(np.arange(Lsys,L))
            pauli_strings[b,:] = (pauli_strings[b,:] + pauli_strings[a,:]) % 2
            pauli_strings[L+a,:] = (pauli_strings[L+a,:] + pauli_strings[L+b,:]) % 2
        
        for i in range(1, Lsys-1, 2):
            twoQubitClif(pauli_strings, allGates[choice(np.arange(576,len(allGates)))], i, i+1)
        for i in range(Lsys+1, L-1, 2):
            twoQubitClif(pauli_strings, allGates[choice(np.arange(576,len(allGates)))], i, i+1)
        for i in range(Lsys):
            if random() > p: continue
            b = choice(np.arange(Lsys,L))
            pauli_strings[b,:] = (pauli_strings[b,:] + pauli_strings[a,:]) % 2
            pauli_strings[L+a,:] = (pauli_strings[L+a,:] + pauli_strings[L+b,:]) % 2
            
        for timesteps in range(max_t):

            gate = np.linalg.matrix_power(pauli_strings, timesteps) % 2
            
            eig1 = (gate - np.eye(2*L, dtype='int8')) % 2
            system = np.vstack((eig1, np.identity(2*L, dtype='int8')))
            system_solved = gf2RowRed(system.T.copy()).T
            

            n_evecs = 0
            eigvecs = []
            for col in range(2*L):
                if np.sum(system_solved[:2*L,col]) == 0:
                    n_evecs += 1
                    eigvec = system_solved[2*L:,col]
                    eigvecs.append(eigvec)
                    if not np.array_equal((gate @ eigvec) % 2, eigvec):
                        print(col, eigvec, (gate @ eigvec) % 2)
                        raise Exception()
            n_evecs_t[timesteps,rep] = n_evecs

            for eigvec in eigvecs:
                ps = np.hstack((eigvec, [0]))

                for t in range(timesteps):
                    floquet = lst_gen(floquet_array)
                    for i in range(0, L ,2):
                        gate = allGates[next(floquet)]
                        twoQubitClif_ps(ps, gate, i, i+1)
                    for i in range(1, L+1, 2):
                        gate = allGates[next(floquet)]
                        twoQubitClif_ps(ps, gate, i, (i+1)%L)

                if (ps[-1] != 0):
                    allphase1[timesteps,rep] = False
                    break
    
    return np.stack((n_evecs_t, allphase1))
    outfile = open(directory + '/L' + str(L) + '_evol' + str(max_t//L) + '_' + name_ext, 'wb')
    pickle.dump(np.stack((n_evecs_t, allphase1)), outfile)
    outfile.close()

In [ ]:
Lsys = 6
Lbath = 30
p = 0.1
reps = 50
max_t = 10
n_evecs_t, allphase1 = exp_sff(Lsys, Lbath, p, reps, max_t)

In [ ]:
print(n_evecs_t.shape)
plt.figure(figsize=(17,4))
plt.subplot(1,2,1)
plt.plot(np.mean(n_evecs_t,axis=1), '.-')
plt.subplot(1,2,2)
plt.plot(np.mean(allphase1,axis=1), '.-')